In [336]:
# used during development to releoad modules every time there is a change
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from risk_kit import Metrics, pre_processing_ind, get_ind30_nfirms,\
                    get_ind30_size, plot_ef2, fixedmix_allocator, glidepath_allocator, floor_allocator, drawdown_allocator
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:.6f}'.format

m = Metrics()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [337]:
# b1 15-Year-bond, FV=1000, 5% coupon, coupons_per_year=2
# b2 5-Year-bond,  FV=1000, 6% coupon, coupons_per_year=4
# b3 10-Year-ZC,   FV=1000
# yield curve flat 5%

b1= m.bond_cash_flows(15, 1000, .05, 2)
b1.shape

(30,)

In [338]:
discounts = m.discount(b1.index, .05/2)
discounts.shape

(30, 1)

In [339]:
b1= m.bond_cash_flows(15, 1000, .05, 2)
discounts = m.discount(b1.index, .05/2)
dcf= discounts*b1.values.reshape(len(b1),1)
#(b1.index*dcf).sum()/dcf.sum()

In [334]:
m.macaulay_duration2(m.bond_cash_flows(15, 1000, .05, 2), .05/2)/2

10.72677495379012

In [335]:
m.macaulay_duration2(m.bond_cash_flows(5, 1000, .06, 4), .05/4)/4

4.373363222636413

In [138]:
m.macaulay_duration2(m.bond_cash_flows(10, 1000, 0., 1), .05)/1

10.0

In [139]:
m.pv(m.bond_cash_flows(15, 1000, .05, 2), 0.05)

0   615.688724
dtype: float64

In [140]:
m.pv(m.bond_cash_flows(5, 1000, .06, 4), 0.05)

0   563.822638
dtype: float64

In [323]:
m.pv(m.bond_cash_flows(10, 1000, 0, 1), 0.05)

0   613.913254
dtype: float64

In [328]:
b1= m.bond_price(15, 1000, .05, 2, 0.05)
b2= m.bond_price(5, 1000, .06, 4, 0.05)
b3= m.bond_price(10, 1000, 0., 1, 0.05)
b1, b2, b3

(0   1000.000000
 dtype: float64,
 0   1043.998290
 dtype: float64,
 0   613.913254
 dtype: float64)

In [142]:
# assume yield flat at 5%. Macaulay Duration?
liab1=100000 # 3 years away
liab2=200000 # 5 years away
liab3=300000 # 10 years away

In [143]:
cf_liab1 = m.bond_cash_flows(3, 100000, 0, 1)
cf_liab2 = m.bond_cash_flows(5, 200000, 0, 1)
cf_liab3 = m.bond_cash_flows(10, 300000, 0, 1)
cf = pd.concat([cf_liab1, cf_liab2, cf_liab3], axis=1)
cf = cf[0].add(cf[1], fill_value=0).add(cf[2], fill_value=0)
cf

1         0.000000
2         0.000000
3    100000.000000
4         0.000000
5    200000.000000
6         0.000000
7         0.000000
8         0.000000
9         0.000000
10   300000.000000
dtype: float64

In [144]:
discounts = m.discount(cf.index, 0.05).squeeze()
discounts

1    0.952381
2    0.907029
3    0.863838
4    0.822702
5    0.783526
6    0.746215
7    0.710681
8    0.676839
9    0.644609
10   0.613913
Name: 0, dtype: float64

In [145]:
# present values of the future cash flows
dcf = discounts*cf
dcf

1         0.000000
2         0.000000
3     86383.759853
4         0.000000
5    156705.233294
6         0.000000
7         0.000000
8         0.000000
9         0.000000
10   184173.976062
dtype: float64

In [146]:
# coverting them into a set of weights
weights = dcf/dcf.sum()
weights

1    0.000000
2    0.000000
3    0.202179
4    0.000000
5    0.366765
6    0.000000
7    0.000000
8    0.000000
9    0.000000
10   0.431055
dtype: float64

In [147]:
# weighted average of all the periods: This is the weighted average time,
#I am waiting to get all my money back
macaulay_duration = (cf.index * weights).sum()
macaulay_duration

6.750917852744651

Assuming the same set of liabilities ,
build a duration matched portfolio of B1 and B2 to match these liabilities.
What is the weight of B2 in the portfolio?
(Hint: the code we developed in match_duration() assumes that all bonds have the same number of coupons per year.
This is not the case here.

In [148]:
liabilities = pd.Series(data=[liab1, liab2, liab3], index=[3, 5, 10])
liabilities

3     100000
5     200000
10    300000
dtype: int64

In [149]:
m.macaulay_duration2(liabilities, 0.05)

6.750917852744651

In [150]:
b1_duration=m.macaulay_duration2(m.bond_cash_flows(15, 1000, .05, 2), .05)/2
b2_duration = m.macaulay_duration2(m.bond_cash_flows(5, 1000, .06, 4), .05)/4
b1_duration, b2_duration

(9.372592076904976, 4.080205545924238)

In [227]:
# create index frequency
data = pd.DataFrame()
data.index = np.arange(60) + 1
data.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [228]:
b1= m.bond_cash_flows(15, 1000, .05, 2)
b1.index =b1.index*2

In [229]:
data = pd.concat([data, b1], axis=1).replace(np.nan, 0).squeeze()
print(data.head())
data.shape
data.iloc[-1]

1    0.000000
2   25.000000
3    0.000000
4   25.000000
5    0.000000
Name: 0, dtype: float64


(60,)

1025.0

In [249]:
p_b1_bond= m.pv(data, 0.05/4)
p_b1_bond

0   996.736445
dtype: float64

In [250]:
b2= m.bond_cash_flows(5, 1000, .06, 4)
b2

1      15.000000
2      15.000000
3      15.000000
4      15.000000
5      15.000000
6      15.000000
7      15.000000
8      15.000000
9      15.000000
10     15.000000
11     15.000000
12     15.000000
13     15.000000
14     15.000000
15     15.000000
16     15.000000
17     15.000000
18     15.000000
19     15.000000
20   1015.000000
dtype: float64

In [251]:
p_b2_bond = m.bond_price(5, 1000, .06, 1, 0.05)
p_b2_bond

0   1043.294767
dtype: float64

In [252]:
short_bond= b2
long_bond=data
#w_s = m.match_duration(liabilities, short_bond, long_bond, 0.05) # discount_rate is 5%

In [296]:
d_t = m.macaulay_duration2(liabilities, 0.05)
d_s = m.macaulay_duration2(short_bond, 0.06/4)/4
d_l = m.macaulay_duration2(long_bond, 0.05/4)/4
w_s = (d_l - d_t)/(d_l - d_s)
w_s

0.6236589170428625

In [297]:
d_t, d_s, d_l, w_s

(6.750917852744651, 4.356542091714541, 10.718791310471271, 0.6236589170428625)

In [299]:
a_0=1000
p_short = m.bond_price(5, 1000, 0.06, 4, 0.05)
p_long=  m.bond_price(15, 1000, 0.05, 2, 0.05)
p_flows =  pd.concat([(a_0*w_s/p_short).values*short_bond, \
                      (a_0*(1-w_s)/p_long).values*long_bond])
duration = m.macaulay_duration2(p_flows, 0.05/4)/4
duration

6.756542027154925

In [264]:
p_short_bond

0   1043.998290
dtype: float64

In [265]:
p_long_bond

0   1000.000000
dtype: float64

In [242]:
(a_0*w_s/p_short_bond).values*short_bond

1     10.066562
2     10.066562
3     10.066562
4     10.066562
5     10.066562
6     10.066562
7     10.066562
8     10.066562
9     10.066562
10    10.066562
11    10.066562
12    10.066562
13    10.066562
14    10.066562
15    10.066562
16    10.066562
17    10.066562
18    10.066562
19    10.066562
20   681.170682
dtype: float64

In [243]:
(a_0*w_s/p_short_bond)

0   0.671104
dtype: float64

In [244]:
p_flows[61:80]
p_flows.shape

42     7.484211
43     0.000000
44     7.484211
45     0.000000
46     7.484211
47     0.000000
48     7.484211
49     0.000000
50     7.484211
51     0.000000
52     7.484211
53     0.000000
54     7.484211
55     0.000000
56     7.484211
57     0.000000
58     7.484211
59     0.000000
60   306.852657
dtype: float64

(80,)

#### b1 15-Year-bond, FV=1000, 5% coupon, coupons_per_year=2
#### b2 5-Year-bond,  FV=1000, 6% coupon, coupons_per_year=4
#### b3 10-Year-ZC,   FV=1000
#### yield curve flat 5%

In [300]:
# combination of b2 and b3
liabilities = pd.Series(data=[liab1, liab2, liab3], index=[3, 5, 10])
liabilities

3     100000
5     200000
10    300000
dtype: int64

In [311]:
# create index frequency
data2 = pd.DataFrame()
data2.index = np.arange(40) + 1
data2.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40],
           dtype='int64')

In [312]:
b3= m.bond_cash_flows(10, 1000, 0., 1)
b3.index =b3.index*4
b3

4       0.000000
8       0.000000
12      0.000000
16      0.000000
20      0.000000
24      0.000000
28      0.000000
32      0.000000
36      0.000000
40   1000.000000
dtype: float64

In [313]:
data2 = pd.concat([data2, b3], axis=1).replace(np.nan, 0).squeeze()
print(data2.head())
data2.shape
data2.iloc[-1]

1   0.000000
2   0.000000
3   0.000000
4   0.000000
5   0.000000
Name: 0, dtype: float64


(40,)

1000.0

In [314]:
p_b3_bond= m.pv(data, 0.05/4)
p_b3_bond

0   608.413335
dtype: float64

In [315]:
b2= m.bond_cash_flows(5, 1000, .06, 4)
b2

1      15.000000
2      15.000000
3      15.000000
4      15.000000
5      15.000000
6      15.000000
7      15.000000
8      15.000000
9      15.000000
10     15.000000
11     15.000000
12     15.000000
13     15.000000
14     15.000000
15     15.000000
16     15.000000
17     15.000000
18     15.000000
19     15.000000
20   1015.000000
dtype: float64

In [316]:
p_b2_bond = m.bond_price(5, 1000, .06, 1, 0.05)
p_b2_bond

0   1043.294767
dtype: float64

In [318]:
short_bond= b2
long_bond=data2

In [320]:
d_t = m.macaulay_duration2(liabilities, 0.05)
d_s = m.macaulay_duration2(short_bond, 0.06/4)/4
d_l = m.macaulay_duration2(long_bond, 0.05/4)/4
w_s = (d_l - d_t)/(d_l - d_s)
w_s

0.5757254151723538

In [322]:
a_0=1000
p_short = m.bond_price(5, 1000, 0.06, 4, 0.05)
p_long =  p_b3_bond
p_flows =  pd.concat([(a_0*w_s/p_short).values*short_bond, \
                      (a_0*(1-w_s)/p_long).values*long_bond])
duration = m.macaulay_duration2(p_flows, 0.05/4)/4
duration

6.760602205328315